In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('./data/sentiment_data_plus.csv')
data

,Sentence,Emotion,From
0,언니 동생으로 부르는게 맞는 일인가요..??,불안,단발성 대화
1,그냥 내 느낌일뿐겠지?,불안,단발성 대화
2,아직너무초기라서 그런거죠?,불안,단발성 대화
3,유치원버스 사고 낫다던데,불안,단발성 대화
4,근데 원래이런거맞나요,불안,단발성 대화
...,...,...,...
171048,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다.,행복,감성대화말뭉치_val
171049,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 ...,불안,감성대화말뭉치_val
171050,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야.,분노,감성대화말뭉치_val
171051,은행 대출이 막혀서 생활비를 구할 수가 없어. 이제 어떻게 살아야 하나 막막해.,분노,감성대화말뭉치_val


In [2]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(data['Emotion'])
num_labels = len(label_encoder.classes_)

data['encoded_label'] = np.asarray(label_encoder.transform(data['Emotion']), dtype=np.int32)
data.head()

,Sentence,Emotion,From,encoded_label
0,언니 동생으로 부르는게 맞는 일인가요..??,불안,단발성 대화,2
1,그냥 내 느낌일뿐겠지?,불안,단발성 대화,2
2,아직너무초기라서 그런거죠?,불안,단발성 대화,2
3,유치원버스 사고 낫다던데,불안,단발성 대화,2
4,근데 원래이런거맞나요,불안,단발성 대화,2


In [3]:
from transformers import BertForSequenceClassification, BertConfig, XLNetTokenizer
from kobert_tokenizer import KoBERTTokenizer
import torch

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
config = BertConfig.from_pretrained('skt/kobert-base-v1')
config.num_labels = num_labels
model = BertForSequenceClassification.from_pretrained('skt/kobert-base-v1', config=config)

model.to('cuda')
torch.cuda.is_available()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at skt/kobert-base-v1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


True

In [15]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=64)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [5]:
from sklearn.model_selection import train_test_split

target = data['encoded_label']
dataset = data.loc[:, ['Sentence', 'encoded_label']]

trainset, test_val_set = train_test_split(dataset,
                                    test_size=0.2,
                                    shuffle=True,
                                    random_state=42,
                                    stratify=target)

# X_train = X_train.reset_index(drop=True)
# X_test_val = X_test_val.reset_index(drop=True)
# y_train = y_train.reset_index(drop=True)
# y_test_val = y_test_val.reset_index(drop=True)

t_target = test_val_set['encoded_label']
testset, validset = train_test_split(test_val_set,
                                   test_size=0.5,
                                   shuffle=True,
                                   random_state=42,
                                   stratify=t_target)

trainset = trainset.reset_index(drop=True)
testset = testset.reset_index(drop=True)
validset = validset.reset_index(drop=True)

In [6]:
tokenizer("일기를 씁시다.", truncation=True, padding=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': [2, 3803, 5573, 517, 6783, 6705, 5782, 54, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
tokenizer.batch_encode_plus("<br>일기를 씁시다")

{'input_ids': [[2, 630, 3], [2, 517, 380, 3], [2, 517, 435, 3], [2, 632, 3], [2, 3803, 3], [2, 1258, 3], [2, 517, 6116, 3], [2, 3], [2, 517, 6783, 3], [2, 2959, 3], [2, 1562, 3]], 'token_type_ids': [[0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0, 0], [0, 0], [0, 0, 0, 0], [0, 0, 0], [0, 0, 0]], 'attention_mask': [[1, 1, 1], [1, 1, 1, 1], [1, 1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1, 1], [1, 1], [1, 1, 1, 1], [1, 1, 1], [1, 1, 1]]}

In [8]:
text = ["안녕하세요", "반갑습니다.", "일기 쓰는 것은 즐겁다. 그렇지만 조금 귀찮아."]

tokenizer(text)

{'input_ids': [[2, 3135, 5724, 7814, 3], [2, 2207, 5345, 6701, 54, 3], [2, 3803, 5561, 3084, 5760, 910, 517, 7313, 5397, 5782, 54, 1205, 6150, 4164, 1174, 7396, 6797, 54, 3]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [9]:
tokenizer.batch_encode_plus(text)

{'input_ids': [[2, 3135, 5724, 7814, 3], [2, 2207, 5345, 6701, 54, 3], [2, 3803, 5561, 3084, 5760, 910, 517, 7313, 5397, 5782, 54, 1205, 6150, 4164, 1174, 7396, 6797, 54, 3]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [17]:
# train_encodings = tokenizer.batch_encode_plus(trainset['Sentence'])
# val_encodings = tokenizer.batch_encode_plus(validset['Sentence'])
# test_encodings = tokenizer.batch_encode_plus(testset['Sentence'])

train_encodings = tokenizer(trainset['Sentence'].tolist(), truncation=True, padding=True, max_length=64, return_tensors="pt")
val_encodings = tokenizer(validset['Sentence'].tolist(), truncation=True, padding=True, max_length=64, return_tensors="pt")
test_encodings = tokenizer(testset['Sentence'].tolist(), truncation=True, padding=True, max_length=64, return_tensors="pt")

In [18]:
import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)
    
train_dataset = Dataset(train_encodings, trainset['encoded_label'].tolist())
val_dataset = Dataset(val_encodings, validset['encoded_label'].tolist())
test_dataset = Dataset(test_encodings, testset['encoded_label'].tolist())

In [22]:
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [23]:
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=val_dataset,
                  compute_metrics=compute_metrics)

trainer.train()

/home/elice/anaconda3/envs/nightsky/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 136842
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 6417


Epoch,Training Loss,Validation Loss,Accuracy
1,0.934300,0.892040,0.681983
2,0.750300,0.870590,0.690810
3,0.584300,0.922618,0.686309


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 17106
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/

TrainOutput(global_step=6417, training_loss=0.7364725044848522, metrics={'train_runtime': 4383.3795, 'train_samples_per_second': 93.655, 'train_steps_per_second': 1.464, 'total_flos': 1.35023472725184e+16, 'train_loss': 0.7364725044848522, 'epoch': 3.0})

In [24]:
trainer.predict(test_dataset)

***** Running Prediction *****
  Num examples = 17105
  Batch size = 8


PredictionOutput(predictions=array([[ 0.06831191,  0.50074655, -0.4283551 , ..., -1.6985977 ,
        -2.2770462 , -1.7191817 ],
       [ 0.29235607,  2.9659004 , -0.33061096, ..., -1.2987227 ,
        -3.127798  ,  3.971012  ],
       [-2.850855  ,  2.7823305 , -2.2580345 , ...,  2.437744  ,
        -2.1444926 ,  1.0928146 ],
       ...,
       [ 0.17570493, -1.2033936 , -3.464555  , ...,  4.362704  ,
        -0.7634575 , -2.957487  ],
       [-1.1810024 ,  4.3056917 , -2.264956  , ...,  1.4193431 ,
        -2.2924385 , -0.29796773],
       [ 1.4102197 , -1.3454844 , -1.9526913 , ...,  5.075554  ,
        -2.5126026 , -2.4439557 ]], dtype=float32), label_ids=array([3, 1, 4, ..., 4, 1, 4]), metrics={'test_loss': 0.9414632320404053, 'test_accuracy': 0.6842443729903537, 'test_runtime': 65.366, 'test_samples_per_second': 261.68, 'test_steps_per_second': 32.723})

In [25]:
trainer.save_model('test_trainer')

Saving model checkpoint to test_trainer
Configuration saved in test_trainer/config.json
Model weights saved in test_trainer/pytorch_model.bin


In [44]:
test_model = BertForSequenceClassification.from_pretrained('/home/elice/nightsky/test_trainer/', local_files_only=True)
test_model.to('cuda')

def prediction(predict_sentence):
  if type(predict_sentence) == str:
      data = [predict_sentence]
  else:
      data = predict_sentence

  inputs = tokenizer(data, return_tensors='pt')
  inputs.to('cuda')

  with torch.no_grad():
        logits = model(**inputs).logits

  predicted_class_id = logits.argmax().item()
  output = model.config.id2label[predicted_class_id]
    
  if output == 0:
    return "공포"
  elif output == 1:
    return "놀람"
  elif output == 2:
    return "분노"
  elif output == 3:
    return "슬픔"
  elif output == 4:
    return "중립"
  elif output == 5:
    return "행복"
  elif output == 6:
    return "혐오"
  return "공포: {0}, 놀람: {1}, 분노: {2}, 슬픔: {3}, 중립: {4}, 행복: {5}, 혐오: {6}".format(logits[0], logits[1], logits[2], logits[3], logits[4], logits[5], logits[6])

loading configuration file /home/elice/nightsky/test_trainer/config.json
Model config BertConfig {
  "_name_or_path": "skt/kobert-base-v1",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "kobert_version": 1.0,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedd

In [45]:
prediction("새로운 모델 학습 코드를 완성했습니다~~")

'LABEL_5'